# Quickstart

## Using the Block Model API Client

Using the notebook utilities provided by `evo-sdk-common`, we can easily interact with the Block Model Service in a Jupyter notebook environment.

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id",
    cache_location="./notebook-data",
).login()

## BlockModelAPIClient

The `BlockModelAPIClient` wraps endpoint functionality to provide a cohesive interface to the underlying API implementation.

In [ ]:
from evo.blockmodels import BlockModelAPIClient

environment = manager.get_environment()
connector = manager.get_connector()

service_client = BlockModelAPIClient(environment, connector, manager.cache)
service_health = await service_client.get_service_health()

print(f"Block Model Service is {service_health.status.name.lower()}")

### Create Block Model

In [ ]:
import pyarrow

from evo.blockmodels.data import RegularGridDefinition
from evo.blockmodels.endpoints.models import RotationAxis

block_grid = RegularGridDefinition(
    model_origin=[0, 0, 0], rotations=[(RotationAxis.x, 20)], n_blocks=[10, 10, 10], block_size=[1, 1, 1]
)
initial_data = pyarrow.table(
    {"i": [1, 2, 3], "j": [4, 5, 6], "k": [7, 8, 9], "column_one": ["A", "D", "E"], "column_two": [1.5, 1.3, 1.2]},
    schema=pyarrow.schema(
        {
            "i": pyarrow.uint32(),
            "j": pyarrow.uint32(),
            "k": pyarrow.uint32(),
            "column_one": pyarrow.string(),
            "column_two": pyarrow.float32(),
        }
    ),
)
block_model, version = await service_client.create_block_model(
    name="My Block Model",
    description="My Block Model",
    grid_definition=block_grid,
    object_path="your/path",
    coordinate_reference_system="EPSG:3395",
    size_unit_id="m",
    initial_data=initial_data,
)

### Add New Columns

In [ ]:
new_cols = pyarrow.table(
    {"i": [1, 1], "j": [4, 4], "k": [6, 7], "column_three": ["C", "D"], "column_four": [4.5, 5.3]},
    schema=pyarrow.schema(
        {
            "i": pyarrow.uint32(),
            "j": pyarrow.uint32(),
            "k": pyarrow.uint32(),
            "column_three": pyarrow.string(),
            "column_four": pyarrow.float32(),
        }
    ),
)

version_with_new_columns = await service_client.add_new_columns(
    bm_id=block_model.id,
    data=new_cols,
    units={"column_four": "g/t"},
)

### Query Block Model as Table

In [ ]:
from evo.blockmodels.endpoints.models import BBox, IntRange

# Select a bounding box to query
# This example selects blocks where: 1 <= i <= 2
bounding_box = BBox(
    i_minmax=IntRange(min=1, max=2),
    j_minmax=IntRange(min=1, max=9),
    k_minmax=IntRange(min=1, max=9),
)

# Table will be a Pyarrow Table
await service_client.query_block_model_as_table(
    bm_id=block_model.id,
    columns=["column_one", "column_four"],
    bbox=bounding_box,
    version_uuid=version_with_new_columns.version_uuid,
)